# AI4PH Data Challenge

Summer 2024

**Yves Nsoga** 
PhD Candidate Biomedical Engineering
University of Calgary
[Data Intelligence for Health Lab](https://cumming.ucalgary.ca/dih)    

In [97]:
# import all packages here 

import pandas as pd
import numpy as np
import cv2

import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [146]:
# Util functions here 
import torch.nn.functional as F
import torch.nn as nn
def stich_image(img_paths: list):
    """stich images with  overlaping to  generate a panoramic view of the POstal  code

    :param img_paths: a list  of image paths
    return panoramic_image
    """
    imgs = []
    for img_path in img_paths:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        imgs.append(img)
    stitchy = cv2.Stitcher.create()
    (dummy, stiched_output) = stitchy.stitch(imgs)
  
    return stiched_output
    

def crop_and_resize(image: np.array, dimension: tuple = (800, 250)):
    """crop  and resize image 

    :param image: Image to resize
    :param dimension:  Expected (width, height) of new image, defaults to (800,250)
    :Returns image: Resized image
    """
    gray_scaled_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray_scaled_img, 0, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(contours, key=cv2.contourArea, reverse=True)
    cnt = cnts[0]
    x, y, w, h = cv2.boundingRect(cnt)
    img_crop = image[y:y+h, x:x+w]
    resized_image = cv2.resize(
        img_crop, dimension, interpolation=cv2.INTER_LINEAR)
    return resized_image

class zone_panorama_dataset(Dataset):
    """
    """
    def __init__(self,csv_file,root_dir,transform=None):
        """_summary_

        :param csv_file: _description_
        :type csv_file: _type_
        :param root_dir: _description_
        :type root_dir: _type_
        :param transform: _description_, defaults to None
        :type transform: _type_, optional
        """
        self.depravation_indices_data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform =transform

    def __len__(self):
        return len(self.depravation_indices_data)
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index =index.tolist()
        image_list = []
        image_path = os.path.join(self.root_dir,
                                  self.depravation_indices_data.iloc[index, 1])
        
        try:
            for images in os.listdir(image_path):
                if (images.endswith(".png")):
                    image_list.append(os.path.join(image_path,
                                                    images))
            panorama_view = stich_image(image_list)
            image = crop_and_resize(panorama_view)
            sample = {"image": image,
                    "SCOREMAT": self.depravation_indices_data.iloc[index, 4], 
                    "SCORESOC": self.depravation_indices_data.iloc[index, 5]}
            if self.transform:
                sample = self.transform(sample)
            return sample
        except IOError:
            pass


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image= sample['image']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        sample['image'] = F.to_tensor(sample['image'])
        # image = image.transpose((2, 0, 1))
        return sample
    

class Net(nn.Module):
    """Convolution  neural network 

    :param nn: _description_
    :type nn: _type_
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [150]:
depravation_dataset = zone_panorama_dataset(
    csv_file="Data challenge dataset_complete_no_duplication.csv", root_dir="GSV_Images_AB", transform=transforms.Compose([ToTensor()]))

In [149]:
fig = plt.figure()
sample = depravation_dataset[130]

for i, sample in enumerate(depravation_dataset):
    if sample:
        plt.figure(figsize=[40, 40])
        ax = plt.subplot(1, 6, i + 1)
        plt.tight_layout()
        ax.set_title(f'Sample #{i} SCOREMAT:{sample["SCOREMAT"]},SCORESOC:{sample["SCORESOC"]}')

        ax.axis('off')
        ax.imshow(sample['image'])

    if i == 5:
        plt.show()
        break

TypeError: __init__() takes 1 positional argument but 2 were given

<Figure size 640x480 with 0 Axes>

In [151]:
#split the dataset into  train and test set
train_set, val_set = torch.utils.data.random_split(
    depravation_dataset, [0.7, 0.3])

In [152]:
train_loader = DataLoader(train_set, batch_size=4,
                                       shuffle=True, num_workers=0)

In [153]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_set, 0):
        # get the inputs; data is a list of [inputs, labels]
        if data != None:
            image = data["image"]
            print(image)
            mat_score = data["SCOREMAT"]

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(image)
            loss = criterion(outputs, mat_score)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

print('Finished Training')

TypeError: __init__() takes 1 positional argument but 2 were given

In [68]:
test = {"A":1,"B":2}
test1,test2 = test
test1

'A'